In [ ]:
from google.colab import auth
auth.authenticate_user()

!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
fileId = drive.CreateFile({'id': '1bjsH1wijxXmXH1quyEpOW1vLgPCZIFZ3'}) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])  # folder_data.zip
fileId.GetContentFile('dataset.zip')  # Save Drive file as a local file

!unzip dataset.zip -d ./

Dataset.zip
Archive:  dataset.zip
   creating: ./Dataset/
   creating: ./Dataset/Train/
   creating: ./Dataset/Train/gl/
  inflating: ./Dataset/Train/gl/Im256.jpg  
  inflating: ./Dataset/Train/gl/Im257.jpg  
  inflating: ./Dataset/Train/gl/Im258.jpg  
  inflating: ./Dataset/Train/gl/Im259.jpg  
  inflating: ./Dataset/Train/gl/Im260.jpg  
  inflating: ./Dataset/Train/gl/Im261.jpg  
  inflating: ./Dataset/Train/gl/Im262.jpg  
  inflating: ./Dataset/Train/gl/Im263.jpg  
  inflating: ./Dataset/Train/gl/Im264.jpg  
  inflating: ./Dataset/Train/gl/Im265.jpg  
  inflating: ./Dataset/Train/gl/Im266.jpg  
  inflating: ./Dataset/Train/gl/Im267.jpg  
  inflating: ./Dataset/Train/gl/Im268.jpg  
  inflating: ./Dataset/Train/gl/Im269.jpg  
  inflating: ./Dataset/Train/gl/Im270.jpg  
  inflating: ./Dataset/Train/gl/Im271.jpg  
  inflating: ./Dataset/Train/gl/Im272.jpg  
  inflating: ./Dataset/Train/gl/Im273.jpg  
  inflating: ./Dataset/Train/gl/Im274.jpg  
  inflating: ./Dataset/Train/gl/Im275.jpg  

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

# Define data directories
train_dir = '/content/Dataset/Train'
test_dir = '/content/Dataset/Test'
val_dir = '/content/Dataset/Validation'

# Image size and batch size
image_size = (224, 224)
batch_size = 16

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load and prepare datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'  # Ensure it's set to 'binary'
)

# Create the EfficientNet model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    model = Model(inputs, outputs, name="U-Net")
    return model

if __name__ == "__main__":
    input_shape = (512, 512, 3)
    model = build_unet(input_shape)
    model.summary()

# optimizer = SGD(learning_rate=0.01)  # Adjust learning rate as needed

model.compile(
    optimizer=Adam(lr=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
callbacks = [early_stopping]

# Train the model
epochs = 20  # Adjust as needed
try:
    # Training code here
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        verbose=1
    )
except Exception as e:
    print("An error occurred during training:", str(e))
    # Handle the error as needed


# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy}')

# Save the model for future use
model.save('glaucoma_detection_model.h5')


Found 280 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d_57 (Conv2D)          (None, 512, 512, 64)         1792      ['input_4[0][0]']             
                                                                                                  
 batch_normalization_54 (Ba  (None, 512, 512, 64)         256       ['conv2d_57[0][0]']           
 tchNormalization)                                                                                
                                                                        

Epoch 1/20
An error occurred during training: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_ob

ValueError: ignored

In [ ]:
import tensorflow as tf

# Check for available GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU Available:", gpus)
else:
    print("No GPU available.")


No GPU available.
